Chapter 3. 신경망
--------

### A. 간단한 신경망 구현

레이어 하나짜리 신경망은 `Y = AX` 로 표현할 수 있음

<img align=left width=200 src="img/3-simple-network.jpg">

In [1]:
import numpy as np

def simple_network(x):
    return x @ np.array([
        [1, 3, 5],
        [2, 4, 6],
    ])

do = lambda *args: (lambda x: print(f'x: {x}\ty: {simple_network(x)}'))(np.array(args))
do(1, 0)
do(0, 1)
do(2, 3)

x: [1 0]	y: [1 3 5]
x: [0 1]	y: [2 4 6]
x: [2 3]	y: [ 8 18 28]


### B. 죠금 더 복잡한 신경망 구현

위에꺼를 여러번 겹치면 레이어 여러개가 된다. 근데 편향을 넣으면 `Y = AX`가 `Y = AX + B`가 됨. 레이어마다 액티베이션 펑션을 넣으면 `Z = f(AX + B)`가 됨.

<img align=left src="img/3-network.jpg">

In [23]:
sigmoid = lambda x: 1/(1 + np.exp(-x))
ident = lambda x: x

def network(X):
    # Network constants
    W1 = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6],
    ])
    B1 = np.array([0.1, 0.2, 0.3])

    W2 = np.array([
        [0.1, 0.4],
        [0.2, 0.5],
        [0.3, 0.6],
    ])
    B2 = np.array([0.1, 0.2])

    W3 = np.array([
        [0.1, 0.3],
        [0.2, 0.4],
    ])
    B3 = np.array([0.1, 0.2])

    # A = XW + B
    Z1 = sigmoid(X@W1 + B1)
    Z2 = sigmoid(Z1@W2 + B2)
    Y = ident(Z2@W3 + B3)
    
    return Y, [X, Z1, Z2, Y]

def do(fn):
    result, layers = fn(np.array([1.0, 0.5]))
    print(f'결과 : {result}')
    print('중간레이어 :')
    print('\n'.join([f'  - {str(layer)}' for layer in layers]))
    print()
    
do(network)

결과 : [ 0.31682708  0.69627909]
중간레이어 :
  - [ 1.   0.5]
  - [ 0.57444252  0.66818777  0.75026011]
  - [ 0.62624937  0.7710107 ]
  - [ 0.31682708  0.69627909]



이때 비동차 벡터를 쓰면 레이어 하나를 어파인 변환(`Y = AX + B`)이 아니라 선형 변환(`Y' = A'X'`)으로 만들 수 있지 않을까?

In [26]:
def network_homogeneous(X_orig):
    # Change input as a homogeneous vector
    X = np.append(X_orig, 1)
    
    # Network constants
    W1 = np.array([
        [0.1, 0.3, 0.5, 0],
        [0.2, 0.4, 0.6, 0],
        [0.1, 0.2, 0.3, 1],
    ])
    W2 = np.array([
        [0.1, 0.4, 0],
        [0.2, 0.5, 0],
        [0.3, 0.6, 0],
        [0.1, 0.2, 1],
    ])
    W3 = np.array([
        [0.1, 0.3, 0],
        [0.2, 0.4, 0],
        [0.1, 0.2, 1],
    ])
    
    # A' = X'W'
    Z1 = sigmoid(X @ W1)
    Z1[-1] = 1
    Z2 = sigmoid(Z1 @ W2)
    Z2[-1] = 1
    Y = ident(Z2 @ W3)    
    
    return Y[0:-1], [X, Z1, Z2, Y]


do(network)
do(network_homogeneous)

결과 : [ 0.31682708  0.69627909]
중간레이어 :
  - [ 1.   0.5]
  - [ 0.57444252  0.66818777  0.75026011]
  - [ 0.62624937  0.7710107 ]
  - [ 0.31682708  0.69627909]

결과 : [ 0.31682708  0.69627909]
중간레이어 :
  - [ 1.   0.5  1. ]
  - [ 0.57444252  0.66818777  0.75026011  1.        ]
  - [ 0.62624937  0.7710107   1.        ]
  - [ 0.31682708  0.69627909  1.        ]



### C. 소프트맥스 구현

출력층에서는 은닉층과 다른 함수를 쓴다. 리그레션에서는 항등함수를, 분류에서는 소프트맥스를 쓴다고함. 소프트맥스를 짜보자.

<!-- https://www.codecogs.com/latex/eqneditor.php -->

$$y_k = \frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)}$$

In [34]:
def softmax_naive(A):
    ExpA = np.exp(A)
    return ExpA / np.sum(ExpA)
    
softmax_naive(np.array([0.3, 2.9, 4.0]))

array([ 0.01821127,  0.24519181,  0.73659691])

이때 분자 부모에 같은 값의 상수를 더해서, 부동소수점 오차를 줄일 수 있다.

$$
\begin{align*}
y_k = \frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)} &= \frac{Cexp(a_k)}{C\sum_{i=1}^{n}exp(a_i)} \\
&= \frac{exp(a_k + logC)}{\sum_{i=1}^{n}exp(a_i + logC)} \\
&= \frac{exp(a_k + C')}{\sum_{i=1}^{n}exp(a_i + C')}
\end{align*}
$$

In [39]:
def softmax(A):
    ExpA = np.exp(A - np.max(A))
    return ExpA / np.sum(ExpA)

# 두 함수의 결과가 같다
print(softmax_naive(np.array([0.3, 2.9, 4.0])))
print(softmax(np.array([0.3, 2.9, 4.0])))

[ 0.01821127  0.24519181  0.73659691]
[ 0.01821127  0.24519181  0.73659691]
